In [1]:
import numpy as np
from scipy.stats import norm

In [5]:
S = 151.03
K = 165
T = (32 / 365)
r = 0.0425
b = 0.53 / 100
sigma = 0.2
N = 100
epsilon = 1e-4

In [6]:
# GBSM
def d1(S, K, r, b, sigma, T):
    return (np.log(S / K) + (b + (sigma ** 2) / 2) * T) / (sigma * np.sqrt(T))

def d2(d1_value, sigma, T):
    return d1_value - sigma * np.sqrt(T)

# Greeks from GBSM
def delta(S, K, r, b, sigma, T, option_type='call'):
    d_1 = d1(S, K, r, b, sigma, T)
    if option_type == 'call':
        return  norm.cdf(d_1) * np.exp((b - r) * T)
    else:
        return (norm.cdf(d_1) - 1) * np.exp((b - r) * T)

def gamma(S, K, r, b, sigma, T):
    d_1 = d1(S, K, r, b, sigma, T)
    return (np.exp((b - r) * T) * norm.pdf(d_1)) / (S * sigma * np.sqrt(T))

def vega(S, K, r, b, sigma, T):
    d_1 = d1(S, K, r, b, sigma, T)
    return S * np.exp((b - r) * T) * norm.pdf(d_1) * np.sqrt(T)

def theta(S, K, r, b, sigma, T, option_type='call'):
    d_1 = d1(S, K, r, b, sigma, T)
    d_2 = d2(d_1, sigma, T)
    if option_type == 'call':
        return - (S * np.exp((b - r) * T) * norm.pdf(d_1) * sigma) / (2 * np.sqrt(T)) - \
               (b - r) * S * np.exp((b - r) * T) * norm.cdf(d_1) - r * K * np.exp(-r * T) * norm.cdf(d_2)
    else:
        return - (S * np.exp((b - r) * T) * norm.pdf(d_1) * sigma) / (2 * np.sqrt(T)) + \
               (b - r) * S * np.exp((b - r) * T) * norm.cdf(-d_1) + r * K * np.exp(-r * T) * norm.cdf(-d_2)

def rho(S, K, r, b, sigma, T, option_type='call'):
    d_2 = d2(d1(S, K, r, b, sigma, T), sigma, T)
    if option_type == 'call':
        return K * T * np.exp(-r * T) * norm.cdf(d_2)
    else:
        return -K * T * np.exp(-r * T) * norm.cdf(-d_2)

def carry_rho(S, K, r, b, sigma, T, option_type='call'):
    d_1 = d1(S, K, r, b, sigma, T)
    if option_type == 'call':
        return T * S * np.exp((b - r) * T) * norm.cdf(d_1)
    else:
        return -T * S * np.exp((b - r) * T) * norm.cdf(-d_1)

greeks = {
    "Delta": (delta(S, K, r, b, sigma, T, 'call'), delta(S, K, r, b, sigma, T, 'put')),
    "Gamma": (gamma(S, K, r, b, sigma, T), gamma(S, K, r, b, sigma, T)),
    "Vega": (vega(S, K, r, b, sigma, T), vega(S, K, r, b, sigma, T)),
    "Theta": (theta(S, K, r, b, sigma, T, 'call'), theta(S, K, r, b, sigma, T, 'put')),
    "Rho": (rho(S, K, r, b, sigma, T, 'call'), rho(S, K, r, b, sigma, T, 'put')),
    "Carry Rho": (carry_rho(S, K, r, b, sigma, T, 'call'), carry_rho(S, K, r, b, sigma, T, 'put'))
}

In [7]:
def option_price(S, K, T, r, b, sigma, option_type='call'):
    d1 = (np.log(S / K) + (b + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'call':
        return S * np.exp((b - r) * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp((b - r) * T) * norm.cdf(-d1)

# Finite difference methods for Greeks
def finite_difference_greeks(S, K, T, r, b, sigma, option_type='call'):
    price_up = option_price(S + epsilon, K, T, r, b, sigma, option_type)
    price_down = option_price(S - epsilon, K, T, r, b, sigma, option_type)
    delta = (price_up - price_down) / (2 * epsilon)

    gamma = (price_up + price_down - 2 * option_price(S, K, T, r, b, sigma, option_type)) / (epsilon ** 2)

    vega = (option_price(S, K, T, r, b, sigma + epsilon, option_type) -
            option_price(S, K, T, r, b, sigma - epsilon, option_type)) / (2 * epsilon)

    theta = (option_price(S, K, T - epsilon, r, b, sigma, option_type) -
             option_price(S, K, T, r, b, sigma, option_type)) / epsilon

    rho = (option_price(S, K, T, r + epsilon, b, sigma, option_type) -
           option_price(S, K, T, r - epsilon, b, sigma, option_type)) / (2 * epsilon)

    carry_rho = (option_price(S, K, T, r, b + epsilon, sigma, option_type) -
                 option_price(S, K, T, r, b - epsilon, sigma, option_type)) / (2 * epsilon)

    return {
        "Delta": delta,
        "Gamma": gamma,
        "Vega": vega,
        "Theta": theta,
        "Rho": rho,
        "Carry Rho": carry_rho
    }

greeks_call = finite_difference_greeks(S, K, T, r, b, sigma, 'call')
greeks_put = finite_difference_greeks(S, K, T, r, b, sigma, 'put')

finite_difference_greeks_list = [
    {"Greek": "Delta", "Call": greeks_call["Delta"], "Put": greeks_put["Delta"]},
    {"Greek": "Gamma", "Call": greeks_call["Gamma"], "Put": greeks_put["Gamma"]},
    {"Greek": "Vega", "Call": greeks_call["Vega"], "Put": greeks_put["Vega"]},
    {"Greek": "Theta", "Call": greeks_call["Theta"], "Put": greeks_put["Theta"]},
    {"Greek": "Rho", "Call": greeks_call["Rho"], "Put": greeks_put["Rho"]},
    {"Greek": "Carry Rho", "Call": greeks_call["Carry Rho"], "Put": greeks_put["Carry Rho"]}
]

In [8]:
# BT
def binomial_tree_american(call, S, K, T, r, q, sigma, N, dividends=None, div_dates=None):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    pu = (np.exp((r - q) * dt) - d) / (u - d)
    pd = 1 - pu
    df = np.exp(-r * dt)
    z = 1 if call else -1

    if dividends and div_dates:
        for div_date in div_dates:
            if div_date < 0 or div_date >= N:
                continue
            S -= dividends[div_dates.index(div_date)]

    option_values = np.maximum(0, z * (S * u ** np.arange(N, -1, -1) * d ** np.arange(0, N + 1) - K))

    for j in range(N - 1, -1, -1):
        for i in range(j + 1):
            option_values[i] = df * (pu * option_values[i + 1] + pd * option_values[i])
            stock_price = S * u ** i * d ** (j - i)
            option_values[i] = max(option_values[i], z * (stock_price - K))

    return option_values[0]

call_value_no_div = binomial_tree_american(True, S, K, T, r, b, sigma, N)
put_value_no_div = binomial_tree_american(False, S, K, T, r, b, sigma, N)

dividend_amounts = [0.88]
dividend_dates = [N - int((4 / 365) * N)]

call_value_with_div = binomial_tree_american(True, S, K, T, r, b, sigma, N, dividend_amounts, dividend_dates)
put_value_with_div = binomial_tree_american(False, S, K, T, r, b, sigma, N, dividend_amounts, dividend_dates)

In [9]:
gbsm_greeks_list = [
    {"Greek": "Delta", "Call": greeks["Delta"][0], "Put": greeks["Delta"][1]},
    {"Greek": "Gamma", "Call": greeks["Gamma"][0], "Put": greeks["Gamma"][1]},
    {"Greek": "Vega", "Call": greeks["Vega"][0], "Put": greeks["Vega"][1]},
    {"Greek": "Theta", "Call": greeks["Theta"][0], "Put": greeks["Theta"][1]},
    {"Greek": "Rho", "Call": greeks["Rho"][0], "Put": greeks["Rho"][1]},
    {"Greek": "Carry Rho", "Call": greeks["Carry Rho"][0], "Put": greeks["Carry Rho"][1]}
]

print("GBSM Greeks:")
print(f"{'Greek':<12} {'Call':<15} {'Put':<15}")
print("-" * 40)
for greek_data in gbsm_greeks_list:
    print(f"{greek_data['Greek']:<12} {greek_data['Call']:<15.6f} {greek_data['Put']:<15.6f}")

print("\nFinite Difference Greeks:")
print(f"{'Greek':<12} {'Call':<15} {'Put':<15}")
print("-" * 40)
for greek_data in finite_difference_greeks_list:
    print(f"{greek_data['Greek']:<12} {greek_data['Call']:<15.6f} {greek_data['Put']:<15.6f}")

american_option_values_list = [
    {"Type": "Without Dividends", "Call": call_value_no_div, "Put": put_value_no_div},
    {"Type": "With Dividends", "Call": call_value_with_div, "Put": put_value_with_div}
]

print("\nAmerican Option Values:")
print(f"{'Type':<18} {'Call Value':<15} {'Put Value':<15}")
print("-" * 48)
for option_data in american_option_values_list:
    print(f"{option_data['Type']:<18} {option_data['Call']:<15.6f} {option_data['Put']:<15.6f}")

GBSM Greeks:
Greek        Call            Put            
----------------------------------------
Delta        0.072398        -0.924346      
Gamma        0.015394        0.015394       
Vega         6.156924        6.156924       
Theta        -7.068720       -5.682323      
Rho          0.933921        -13.478033     
Carry Rho    0.958620        -12.239253     

Finite Difference Greeks:
Greek        Call            Put            
----------------------------------------
Delta        0.072398        -0.924346      
Gamma        0.015394        0.015390       
Vega         6.156924        6.156924       
Theta        -7.066234       -5.679833      
Rho          -0.024699       -1.238780      
Carry Rho    0.958620        -12.239253     

American Option Values:
Type               Call Value      Put Value      
------------------------------------------------
Without Dividends  0.581716        20.791972      
With Dividends     0.461922        21.628929      
